# Customer Segmentation - K-Means Analysis

# Libraries imports and first insight

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [3]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

First let's explore the dataset first

In [ ]:
df = pd.read_csv(r'../input/Mall_Customers.csv')
df.head()

The features are quite explicit.

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

No need to clean the dataset

In [ ]:
df.describe()

---

# Data exploration and visualization

Plot pairwise relationships between features in a dataset.

In [ ]:
plt.figure(1, figsize=(16,10))
sns.pairplot(data=df, hue='Gender')
plt.show()

Number of male vs female

In [ ]:
plt.figure(1, figsize=(4,4))
sns.countplot(x='Gender', data=df)
plt.show()

Distribution of numerical features (Age, Annual income & Spending score)

In [ ]:
plt.figure(1, figsize=(16,4))
n = 0
for x in ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']:
    n += 1
    plt.subplot(1, 3, n)
    plt.subplots_adjust(hspace=0.5 , wspace=0.5)
    sns.distplot(df[x] , bins=10)
    plt.title('Distplot of {}'.format(x))
plt.show()

---

# Clustering using K- means

## ML model

__Concept__

K-means clustering is one of the simplest and popular unsupervised machine learning algorithms.

Typically, unsupervised algorithms make inferences from datasets using only input vectors without referring to known, or labelled, outcomes.

A cluster refers to a collection of data points aggregated together because of certain similarities.

You’ll define a target number k, which refers to the number of centroids you need in the dataset. A centroid is the imaginary or real location representing the center of the cluster.

Every data point is allocated to each of the clusters through reducing the in-cluster sum of squares.

In other words, the K-means algorithm identifies k number of centroids, and then allocates every data point to the nearest cluster, while keeping the centroids as small as possible.

The ‘means’ in the K-means refers to averaging of the data; that is, finding the centroid.

__How the K-means algorithm works__

To process the learning data, the K-means algorithm in data mining starts with a first group of randomly selected centroids, which are used as the beginning points for every cluster, and then performs iterative (repetitive) calculations to optimize the positions of the centroids

It halts creating and optimizing clusters when either:

The centroids have stabilized — there is no change in their values because the clustering has been successful.
The defined number of iterations has been achieved.

__Optimal K: the elbow method__

How many clusters would you choose ?


A common, empirical method, is the elbow method. You plot the mean distance of every point toward its cluster center, as a function of the number of clusters.

Sometimes the plot has an arm shape, and the elbow would be the optimal K.

<img src="input/elbow.png" style="height:300px">

Warning: this method does not apply all the time: sometimes you don't have a clear elbow! In any case, you have to check on the data how is the clustering and make sure it makes sense.

## Application in this use-case

Let's perform clustering (optimizing K with the elbow method). In order to simplify the problem, we start by keeping only the two last columns as features.

In [ ]:
X = df.iloc[:, -2:]

In [ ]:
km_inertias, km_scores = [], []

for k in range(2, 11):
    km = KMeans(n_clusters=k).fit(X)
    km_inertias.append(km.inertia_)
    km_scores.append(silhouette_score(X, km.labels_))

sns.lineplot(range(2, 11), km_inertias)
plt.title('elbow graph / inertia depending on k')
plt.show()

In [ ]:
sns.lineplot(range(2, 11), km_scores)
plt.title('scores depending on k')
plt.show()

Now let's apply K-means on more than 2 features.

In [ ]:
X = df.iloc[:, -3:]

In [ ]:
km_inertias, km_scores = [], []

for k in range(2, 11):
    km = KMeans(n_clusters=k).fit(X)
    km_inertias.append(km.inertia_)
    km_scores.append(silhouette_score(X, km.labels_))

sns.lineplot(range(2, 11), km_inertias)
plt.title('elbow graph / inertia depending on k')
plt.show()

In [ ]:
sns.lineplot(range(2, 11), km_scores)
plt.title('scores depending on k')
plt.show()

In [ ]:
km = KMeans(n_clusters=5).fit(X)

In [ ]:
# K-Means visualization on pair of 2 features
plt.figure(figsize=(10, 6))
sns.scatterplot(X.iloc[:, 1], X.iloc[:, 2], hue=km.labels_)
plt.show()

In [ ]:
# K-Means visualization on another pair of 2 features
plt.figure(figsize=(10, 6))
sns.scatterplot(X.iloc[:, 0], X.iloc[:, 1], hue=km.labels_)
plt.show()

In [ ]:
# K-Means visualization on the last pair of 2 features
plt.figure(figsize=(10, 6))
sns.scatterplot(X.iloc[:, 0], X.iloc[:, 2], hue=km.labels_)
plt.show()

## Visualization  of the clusters in a 3D scatter plot.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(8,8))
ax = Axes3D(fig)

xs = X.iloc[:, 0]
ys = X.iloc[:, 1]
zs = X.iloc[:, 2]
ax.scatter(xs, ys, zs, s=50, alpha=0.6, c=km.labels_)

ax.set_xlabel('Age')
ax.set_ylabel('Annual income')
ax.set_zlabel('Spending score')

plt.show()

This Clustering Analysis gives us a very clear insight about the different segments of the customers in the Mall. There are clearly 5 segments of Customers based on their Annual Income and Spending Score which are reportedly the best factors/attributes to determine the segments of a customer in a Mall.

## Definition of customers profiles corresponding to each clusters

In [ ]:
# Profiles of customers
X['label'] = km.labels_

In [ ]:
X.label.value_counts()

In [ ]:
for k in range(5):
    print(f'cluster nb : {k}')
    print(X[X.label == k].describe().iloc[[0, 1, 3, 7], :-1])
    print('\n\n')

In [ ]:
X[X.label == 1].describe().iloc[[0, 1, 3, 7], :-1]

---

In [ ]:
# Cluster counts
cluster_counts = [79, 36, 39, 23, 23]

# Total count
total_count = sum(cluster_counts)

# Calculate percentages
cluster_percentages = [(count / total_count) * 100 for count in cluster_counts]
cluster_percentages


## Key Findings:
- **Cluster 0**: High-income customers with **moderate spending** (11.5% of the dataset).  
  **Impact**: This group represents potential high-value customers, where targeted promotions could boost spending, increasing overall revenue.
  
- **Cluster 1**: Customers with **low spending** and **medium income** (3.6% of the dataset).  
  **Impact**: These customers are underperforming. Special engagement strategies like personalized offers or discounts could help increase their spending.
  
- **Cluster 2**: Customers with **high spending** and **low income** (18.8% of the dataset).  
  **Impact**: This group is spending a lot but may need financial management tools or loyalty programs to maintain their engagement. They could benefit from targeted communication about affordable options or budgeting advice.
  
- **Cluster 3**: Young customers with **low spending** (2.3% of the dataset).  
  **Impact**: This group may be in the early stages of brand interaction. Offering them incentives like discounts or rewards for their first purchase could increase their lifetime value.
  
- **Cluster 4**: Older customers with **low income** and **moderate spending** (18.8% of the dataset).  
  **Impact**: These customers might benefit from budget-friendly offers or loyalty programs that can increase their frequency of purchase, helping to maximize customer retention.
